In [28]:
import pandas as pd
from transformers import TrOCRProcessor, ConditionalDetrConfig, Trainer, TrainingArguments
from PIL import Image
import os

In [55]:
import pandas as pd
from transformers import TrOCRProcessor, ConditionalDetrConfig, Trainer, TrainingArguments
from PIL import Image
from tqdm import tqdm
import shutil
import torch
import os

# Cargar el CSV
csv_file = r'data\output\divide_images_train\finetuning.csv'
df = pd.read_csv(csv_file)

# Eliminar filas donde el texto está vacío
df = df.dropna(subset=['text'])

# Definir el procesador y el modelo preentrenado
processor = TrOCRProcessor.from_pretrained('qantev/trocr-large-spanish')
model = ConditionalDetrConfig.from_pretrained('qantev/trocr-large-spanish')

# Configurar los datos de entrenamiento
train_texts = df['text'].tolist()  # Usamos 'text' como la descripción del texto en el CSV
train_images = df['image_dir'].tolist()  # Usamos 'image_dir' como la ruta a las imágenes en el CSV

# Función para preprocesar los datos
def preprocess_function(examples):
    pixel_values = []
    progress_bar = tqdm(total=len(examples['image_dir']), desc="Procesando imágenes", position=0, leave=True)
    for image_path, text in zip(examples['image_dir'], examples['text']):
        try:
            # Abrir la imagen con PIL
            with Image.open(image_path) as image:
                # Convertir la imagen a un tensor de PyTorch
                image_tensor = torch.tensor(np.array(image))  # Convertir la imagen a un numpy array y luego a tensor
                image_tensor = image_tensor.permute(2, 0, 1)  # Asegurar el orden correcto de las dimensiones
                
                # Procesar la imagen con el TrOCRProcessor
                processed_input = processor(image_tensor, return_tensors="pt")
                pixel_values.append(processed_input.pixel_values)
        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")
        finally:
            # Actualizar la barra de progreso
            progress_bar.update(1)
            progress_bar.set_postfix({"Texto": text})
    
    progress_bar.close()

    targets = processor(examples['text'], return_tensors="pt").input_ids
    return {
        'pixel_values': pixel_values,
        'labels': targets,
    }

# Preprocesar los datos
train_dataset = preprocess_function({'image_dir': train_images, 'text': train_texts})

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=500,
)

# Inicializar el Trainer para el fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=processor,
)

# Fine-tuning del modelo
trainer.train()

# Guardar el modelo finetuneado y el procesador
trainer.save_model(r'models\modelo_finetuneado_trOCR')
processor.save_pretrained(r'models\procesador_finetuneado_trOCR')

# Borrar el directorio 'microsoft/trocr-base-handwritten'
shutil.rmtree('microsoft/trocr-base-handwritten')

print("Eliminado el directorio 'microsoft/trocr-base-handwritten' después de completar el proceso.")

# Ruta donde se almacena la caché (reemplaza 'ruta_de_la_cache' con la ruta real)
cache_path = r'C:\Users\Jack\.cache\huggingface\hub\models--microsoft--trocr-large-handwritten'

# Verificar si la ruta existe y es un directorio
if os.path.exists(cache_path) and os.path.isdir(cache_path):
    # Borrar todos los archivos y subdirectorios dentro de la caché
    shutil.rmtree(cache_path)
    print(f"Caché en {cache_path} borrada correctamente.")
else:
    print(f"No se encontró la caché en {cache_path}.")



e:\ANACONDA\envs\SATURDAYS-2\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type vision-encoder-decoder to instantiate a model of type conditional_detr. This is not supported for all configurations of models and can yield errors.
Procesando imágenes: 100%|██████████| 131/131 [00:16<00:00,  8.07it/s, Texto=NO HE TENIDO NI QUE USAR MIS ARMAS]                                                                                                                                                                                                                                                                                           


ValueError: Invalid image type. Expected either PIL.Image.Image, numpy.ndarray, torch.Tensor, tf.Tensor or jax.ndarray, but got <class 'list'>.